## PageRank & HITS(Hyperlink-Induced Topic Search)

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext

# create the Spark Session
spark = SparkSession.builder.appName("spark").getOrCreate()

# create the Spark Context
sc = spark.sparkContext

/home/ljj0512/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [33]:
data = sc.textFile("graph-full.txt")
print(data.take(3))
print("# edges:",data.count())
data = data.distinct()
print("# real edges (eliminate dup edges):",data.count())

check = data.map(lambda line: (line.split("\t")[0], 1)).reduceByKey(lambda x, y: x+y)
num_node = check.count()
print("# node:", num_node)

check2 = data.filter(lambda line: line.split("\t")[1] == '1')
print(check2.take(11))
print(check2.count())

['1\t2', '2\t3', '3\t4']
# edges: 8192
# real edges (eliminate dup edges): 8161
# node: 1000
['1000\t1', '894\t1', '686\t1', '913\t1', '398\t1', '488\t1', '55\t1', '101\t1', '227\t1', '251\t1', '168\t1']
11


In [48]:
graph = data.map(lambda line: (int(line.split("\t")[0]), [int(line.split("\t")[1])]) )\
            .reduceByKey(lambda x, y: x+y)\
                .sortByKey(ascending=False)
print(graph.take(3))
print(graph.count())

[(1000, [1, 476, 223, 729, 478, 751, 713, 917, 756, 108, 429, 255, 91, 548]), (999, [1000, 166, 97, 436, 29, 106, 545, 561, 718, 906, 938, 381, 744]), (998, [236, 647, 999, 381, 758, 641, 516, 965])]
1000


In [49]:
graph = data.map(lambda line: (int(line.split("\t")[0]), [int(line.split("\t")[1])]) )\
            .reduceByKey(lambda x, y: x+y)\
                .sortByKey()
print(graph.take(3))
print(graph.count())

[(1, [904, 689, 2, 586, 502, 531]), (2, [799, 415, 690, 632, 440, 498, 3, 505, 781, 713, 433]), (3, [4, 981, 190, 545, 562, 796, 679, 455, 619])]
1000


In [57]:
import numpy as np

# M = np.matrix([ np.repeat(0,num_node) for _ in range(num_node) ])
M = np.matrix(np.zeros((num_node, num_node), dtype = np.float))

count = 0
for i,(start,edges) in enumerate(graph.collect()):
    n = len(edges)
    for edge in edges:
        M[edge-1,i] = 1/n

In [58]:
import numpy as np
r = np.repeat(1/num_node, num_node).reshape(1,-1).T
print(r.shape)
print(r[0])
beta = 0.8
iteration = 40

(1000, 1)
[0.001]


In [59]:
for _ in range(iteration):
    r = ((beta)*M@r) + ((1-beta)/num_node)

In [60]:
print(r)

[[0.00114126]
 [0.00081032]
 [0.00073449]
 [0.00161361]
 [0.00113683]
 [0.00128746]
 [0.00111545]
 [0.00100457]
 [0.00085827]
 [0.00145924]
 [0.00092707]
 [0.00079051]
 [0.00074203]
 [0.00072442]
 [0.00109081]
 [0.00179165]
 [0.00108482]
 [0.00100167]
 [0.00067958]
 [0.00105333]
 [0.00120588]
 [0.00078686]
 [0.00090348]
 [0.00073999]
 [0.00078159]
 [0.00064479]
 [0.00123237]
 [0.00105735]
 [0.00094836]
 [0.00082036]
 [0.00073574]
 [0.00125578]
 [0.00073175]
 [0.00087024]
 [0.0011502 ]
 [0.00066473]
 [0.00069661]
 [0.00055652]
 [0.00121579]
 [0.00103709]
 [0.00075403]
 [0.00121014]
 [0.0012753 ]
 [0.00079432]
 [0.00100303]
 [0.0012449 ]
 [0.00085077]
 [0.00156903]
 [0.00109996]
 [0.00052871]
 [0.000934  ]
 [0.00086908]
 [0.00124912]
 [0.00128154]
 [0.00081691]
 [0.00150668]
 [0.00112692]
 [0.00111286]
 [0.00091431]
 [0.00089602]
 [0.00114566]
 [0.00035315]
 [0.0005708 ]
 [0.00045994]
 [0.00085509]
 [0.0011283 ]
 [0.00064858]
 [0.00057769]
 [0.00081708]
 [0.00065419]
 [0.00103249]
 [0.00

In [61]:
r1 = np.array(r)

for _ in range(5):
    # print(r.min())
    print(r.argmin()+1,"",r1.min())
    r[r.argmin()] = 1.


558  0.0003286018525215297
93  0.0003286018525215297
62  0.0003286018525215297
424  0.0003286018525215297
408  0.0003286018525215297


In [62]:
for _ in range(5):
    # print(r.max())
    print(r1.argmax()+1,"",r1.max())
    r1[r1.argmax()] = 0.



263  0.002020291181518219
537  0.00194334157145315
965  0.0019254478071662631
243  0.0018526340162417312
285  0.0018273721700645142


In [72]:
graph = data.map(lambda line: (int(line.split("\t")[0]), [int(line.split("\t")[1])]) )\
            .reduceByKey(lambda x, y: x+y)\
                .sortByKey()\
                    .map(lambda line: line[1])
print(graph.take(2))
flat = graph.flatMap(lambda x: x).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).map(lambda x: (x[1], x[0]))\
                .sortByKey(ascending=False)

## 이 결과를 통해서 단순히 in flow가 많은 edge가 가장 중요도가 놓은 page는 아니라는 것을 알 수 있다.
## (flow가 많아질 수록 그 가치를 판단하는 것일 복잡해지기 때문에)
## 반면에 가장 적은 중요도를 갖는 edge들은 in flow와 거의 비례함을 확인할 수 있다.
for i in flat.collect():
    print(i[1])

[[904, 689, 2, 586, 502, 531], [799, 415, 690, 632, 440, 498, 3, 505, 781, 713, 433]]
502
16
146
130
494
624
126
799
533
387
243
747
965
893
473
198
298
736
48
108
56
738
781
679
537
977
647
999
857
861
4
986
10
812
940
144
434
568
314
780
280
78
90
994
98
192
362
608
166
846
636
389
263
381
367
983
659
303
373
105
511
285
749
525
853
215
187
445
809
359
690
662
924
552
692
170
566
42
682
188
664
354
606
798
276
792
538
148
218
266
186
716
962
931
603
909
947
763
915
27
61
143
123
497
295
227
515
53
257
149
735
819
91
697
361
113
441
351
597
189
255
993
774
560
128
320
864
366
984
882
968
96
116
572
344
46
936
668
252
336
194
76
484
184
828
580
400
312
912
390
876
162
326
368
768
852
670
415
981
967
637
573
587
667
805
35
127
21
369
811
337
495
43
131
767
89
959
591
1
301
655
875
201
871
883
715
835
211
841
357
481
904
586
536
6
746
964
66
734
358
306
718
250
820
340
432
428
342
58
284
278
490
918
844
200
582
32
544
794
216
894
40
832
158
212
254
232
588
822
224
468
472
100
348
220
152